# Text Generation (Char-RNN) One-to-Many
On this example we will generate text by feeding one character and let the RNN network generate the text. The example uses the one-to-may structure.

### Training
The training process is just the training of a character level languange model, where we input a sequence and expect the same sequence in order to learn the details of a certain language. During training this will be the many-to-many architecture.

#### What is a Language Model
It's basically a network that gives the probability of a sequence, in other words, it gives the probability of s string of being part of some language. Language models are used for example to correct speach recognition systems.
$$P(\text{"The apple and bear salad"})=0.01$$
$$P(\text{"The apple and pear salad"})=0.4$$

### Evaluation
During evaluation we simply give some input to the RNN, sample it's output at random and feed this output back again to the RNN.

### Referenes
* http://karpathy.github.io/2015/05/21/rnn-effectiveness/
* https://www.youtube.com/watch?v=CKrxdgqBheY
* https://github.com/Kulbear/deep-learning-coursera/blob/master/Sequence%20Models/Dinosaurus%20Island%20--%20Character%20level%20language%20model%20final%20-%20v3.ipynb
* https://github.com/furkanu/deeplearning.ai-pytorch/blob/master/5-%20Sequence%20Models/Week%201/Dinosaur%20Island%20--%20Character-level%20language%20model/Dinosaur%20Island%20RNN.ipynb
* https://medium.com/@ppasumarthi_69210/language-model-using-char-rnn-1df53f735880
* https://medium.com/@jianqiangma/all-about-recurrent-neural-networks-9e5ae2936f6e
* https://medium.com/datathings/the-magic-of-lstm-neural-networks-6775e8b540cd
* https://medium.com/phrasee/neural-text-generation-generating-text-using-conditional-language-models-a37b69c7cd4b
* https://medium.com/@florijan.stamenkovic_99541/rnn-language-modelling-with-pytorch-packed-batching-and-tied-weights-9d8952db35a9